# Example for automated image genration using orthophtos and shape-files


## 1.) Input files:
- specify file paths
- choose webdriver (tested with chrome,firefox,safrie). (Default=chrome, download via: https://chromedriver.chromium.org/downloads, copy file to: copy to usr/local/bin/


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


script_dir = '/Users/flo/Proj/sat_hedge_detection/Examples/'
shape_file_path = '../data/Biotopkartierung/Biotopkartierung.shp'
wmts = "http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg"

## choose driver
args_driver = 'chrome'
args_driver_path = '/usr/local/bin/chromedriver'


print("Done")

Done


## 2.) Load packages and functions



In [2]:
import os
import io
from PIL import Image

import geopandas as gpd

import folium
from folium.folium import Map
from folium.features import DivIcon

from pyproj import Transformer

print('Start...')

def getText(temp_biotop):
    temp_nummer = temp_biotop['Nummer'][temp_biotop['Nummer'].keys()[0]]
    temp_gemnr = temp_biotop['gemnr'][temp_biotop['gemnr'].keys()[0]]
    temp_biotoptyp = temp_biotop['biotoptyp'][temp_biotop['biotoptyp'].keys()[0]]
    temp_bezeich = temp_biotop['bezeich'][temp_biotop['bezeich'].keys()[0]]
    text = "Nummer: " + str(temp_nummer) + "\n" + "GemNr: " + str(temp_gemnr) + "\n" + "Biotoptyp: " + str(
        temp_biotoptyp) + "\n" + "Bezeichnung: " + str(temp_bezeich)
    return text

def style_fcn(x):
    return {'stroke':True,'color': '#FF0000', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}

def style_fnc_mask(x):
    return {'stroke':True,'color': '#FFFFFF', 'fillColor': '#FFFFFF','opacity':1, 'weight':10, 'line_cap':'round', 'fill':True, 'fillOpacity': 1}

def style_fcn_hull(x):
    return {'stroke':True,'color': '#008FFF', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}


def style_fcn_env(x):
    return {'stroke':True,'color': '#F700FF', 'fillColor': '#AARRGGBB','opacity':1, 'weight':2, 'line_cap':'round', 'fill':False}

def create_info_textfile(biotop):

    temp_nummer = temp_biotop['Nummer'][temp_biotop['Nummer'].keys()[0]]
    temp_gemnr = temp_biotop['gemnr'][temp_biotop['gemnr'].keys()[0]]
    temp_biotoptyp = temp_biotop['biotoptyp'][temp_biotop['biotoptyp'].keys()[0]]
    temp_bezeich = temp_biotop['bezeich'][temp_biotop['bezeich'].keys()[0]]
    temp_bio_katgem = temp_biotop['bio_katgem'][temp_biotop['bio_katgem'].keys()[0]]
    temp_bio_subnr = temp_biotop['bio_subnr'][temp_biotop['bio_subnr'].keys()[0]]
    temp_bezeichnun = temp_biotop['bezeichnun'][temp_biotop['bezeichnun'].keys()[0]]
    temp_rechtl_Sch = temp_biotop['rechtl_Sch'][temp_biotop['rechtl_Sch'].keys()[0]]
    link1 = temp_biotop['Link'][temp_biotop['Link'].keys()[0]]
    link2 = temp_biotop['Link_ext'][temp_biotop['Link_ext'].keys()[0]]
    #link1: http://anwendung/ins/biotop/display.do?id=1175590
    #link2: https://service.salzburg.gv.at/ins/biotop/disp...
    # =>    https://service.salzburg.gv.at/ins/biotop/ + display.do?id=1175590
    # link immer 7-Stellig? Sonst nach = suchen
    ID = link1[-7:]
    link_public = "https://service.salzburg.gv.at/ins/biotop/display.do?id="+str(ID)
    link_intern = "https://portal.salzburg.gv.at/ins/biotop/display.do?id="+str(ID)

    temp_path = args_output_dir + 'bio_' + temp_nummer + '/' + temp_nummer + "_info.txt"

    output_file = open(temp_path, 'w')
    output_file.write("Biotop Nummer: "+ str(temp_nummer) + "\n")
    output_file.write("Biotop Kat Gem: "+ str(temp_bio_katgem) + "\n")
    output_file.write("Biotop Sub Nr: "+ str(temp_bio_subnr) + "\n")
    output_file.write("Gemeinde Nummer: "+ str(temp_gemnr)+ "\n")
    output_file.write("Biotop Typ: "+ str(temp_biotoptyp) + "\n")
    output_file.write("Biotop Bezeichnung: "+ str(temp_bezeich) + "\n")
    output_file.write("Biotop Bezeichnung(2): "+ str(temp_bezeichnun) + "\n")
    output_file.write("Biotop §: "+ str(temp_rechtl_Sch) + "\n")
    output_file.write("Biotop Link-Public: "+ link_public + "\n")
    output_file.write("Biotop Link-Public: "+ link_intern + "\n")
    output_file.write("Biotop Location: "+ str(biotop_center(biotop)) + "\n")
    output_file.write("Biotop Surface (approx m2): {:02.2f}\n".format(biotop_surface(biotop)))
    output_file.close()
    
def biotop_surface(biotop):
    #check?
    temp = temp_biotop['geometry'].to_crs("EPSG:3395")\
               .map(lambda p: p.area)
    return temp[temp.keys()[0]]*0.450


def biotop_center(temp_biotop):
    #get biotop pos
    center_temp = temp_biotop.geometry.centroid
    key = center_temp.keys()[0]
    return transformer.transform(center_temp[key].x,center_temp[key].y)

def biotop_current_map(temp_location, text, BIOTOP_BORDER, BIOTOP_MASK ,BIOTOP_DESCRIPTION, case):
    BIOTOP_SCALE = BIOTOP_CONVEX = BIOTOP_ENV = 0

    m_temp = Map(tiles=None, location=[temp_location[1], temp_location[0]],
                 zoom_start=20,
                 prefer_canvas=True,
                 no_touch=True,
                 disable_3d=True,
                 attr="test",
                 control_scale=True,
                 zoom_control=False)

    atlas = folium.raster_layers.WmsTileLayer(url=wmts, layers="bmaporthofoto30cm",
                                              fmt='image/png',
                                              transparent=True,
                                              overlay=True).add_to(m_temp)

    if BIOTOP_BORDER:
        case = 'B_'+str(case)
        folium.GeoJson(temp_biotop, style_function=style_fcn).add_to(m_temp)
    # folium.GeoJson(temp_biotop_scaled,style_function=style_fcn).add_to(m_temp)
    # folium.GeoJson(temp_biotop_convex_hull, style_function=style_fcn_hull).add_to(m_temp)
    # folium.GeoJson(temp_biotop_envelope, style_function=style_fcn_env).add_to(m_temp)

    if BIOTOP_MASK:
        case = 'M_'+str(case)
        folium.GeoJson(temp_biotop, style_function=style_fnc_mask).add_to(m_temp)

    ## WRITE BIOTOP DESCIBTION IN IMAGE
    if BIOTOP_DESCRIPTION:
        folium.map.Marker([temp_location[1] + 0.001, temp_location[0] - 0.001],
                    icon=DivIcon(
                        icon_size=(150, 36),
                        icon_anchor=(0, 0),
                        html='<div style="font-size: 10pt;"><span style="color: #ff0000;">%s</span></div>' % text)).add_to(m_temp)
    if BIOTOP_SCALE:
        temp_biotop_scaled = temp_biotop.scale(xfact=2, yfact=2, zfact=0, origin='center')
    if BIOTOP_CONVEX:
        temp_biotop_convex_hull = temp_biotop.convex_hull
    if BIOTOP_ENV:
        temp_biotop_envelope = temp_biotop.envelope
    save_current_biotop2(m_temp, bio_i, case)
    #return m_temp

def save_current_biotop(m_temp, bio_number):
    png = m_temp._to_png()
    out = open(os.path.join(script_dir, '../data/output_biotop/'+bio_number+'.png'), 'wb')
    bytes_written = out.write(png)
    out.close()

def create_bio_path(bio_i):
    temp_path = '../data/output_biotop_dir/' + 'bio_' + bio_i + '/'
    if not os.path.exists(temp_path):
        os.makedirs(temp_path)
        return True
    return False

def save_current_biotop2(m_temp, bio_number,case):
    img_data = m_temp._to_png()
    img = Image.open(io.BytesIO(img_data))
    temp_path = args_output_dir + 'bio_' + bio_number + '/'
    img.save(os.path.join(script_dir, temp_path + case + '_' + bio_number + '.png'))
    
    
print('Load: Biotopkartierung ', shape_file_path)
nReserve = gpd.read_file(os.path.join(script_dir, shape_file_path))

print('Load: wmts layer:' ,wmts)

if args_driver == 'chrome':
    print('Start: webdriver ', args_driver)
    #driver = webdriver.Chrome(executable_path=args_driver_path)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(args_driver_path, options=options)
else:
    print('Different webdriver... break')#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')


print("Done")

Start...
Load: Biotopkartierung  ../data/Biotopkartierung/Biotopkartierung.shp
Load: wmts layer: http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg
Start: webdriver  chrome
Done


# 3.) Main script
- generates images (png) based on orthophtos and biotop shapefile 
     currently: biotoptyp=='8.1.1.1 + biotoptyp=='8.1.1.2
- create path: sat_hedge_detection/data/output_biotop_dir/

### Parameter to specify:
-  currently: todo

## 3.1) Extract all Biotope regions

In [3]:
#script_dir = os.path.dirname(__file__)
#script_dir = '/Users/flo/Proj/sat_hedge_detection/src/bio_generator'

args_shapefile = '../data/Biotopkartierung/Biotopkartierung.shp'
args_biotypes = ['8.1.1.1', '8.1.1.2']
args_wmts = 'http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg'
args_text = True
args_output_dir = '../data/output_biotop_dir/'


biotop_dict = {}
for i in range(0,len(args_biotypes)):
    str_nummer = "\'" + args_biotypes[i] + "\'"
    query_str = "biotoptyp==" + str(str_nummer)
    biotop_dict[i] = nReserve.query(query_str)

print('Load: bmaporthofoto30cm')
wmts = args_wmts
#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')

if args_driver == 'chrome':
    print('Start: webdriver ', args_driver)
    #driver = webdriver.Chrome(executable_path=args_driver_path)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(args_driver_path, options=options)
else:
    print('Different webdriver... break')


transformer = Transformer.from_crs(31258, 4326, always_xy=True)  # 1frpm 2to


len_totoal = old_i =  0
for iter_bio in range(0,len(args_biotypes)):
    len_totoal = len_totoal + biotop_dict[iter_bio].shape[0]

#for biotop_dict[iter_bio] in biotop_dict:
for iter_bio in range(0,len(args_biotypes)):
    biotop_key_nummer = biotop_dict[iter_bio]['Nummer']
    biotop_keys = biotop_key_nummer.keys()
    for i in range(0, biotop_dict[iter_bio].shape[0]-1):
        key_i = biotop_keys[i]
        bio_i = biotop_dict[iter_bio]['Nummer'][key_i]
        str_nummer = "\'" + bio_i + "\'"
        query_str = "Nummer==" + str(str_nummer)
        temp_biotop = biotop_dict[iter_bio].query(query_str)
        temp_location = biotop_center(temp_biotop)

        PATH_NEW = create_bio_path(bio_i)
        if PATH_NEW:
            print("Process Biotop: ", i+old_i, "/", len_totoal, " ", bio_i, " type: ", args_biotypes[iter_bio])
            #m_temp = biotop_current_map(temp_location,text=getText(temp_biotop),BIOTOP_BORDER=False, BIOTOP_DESCRIPTION=False, case=1)
            
            ## Raw Bio Image
            biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=True,
                        case=args_biotypes[iter_bio][-1])
            
            ## Red Border File
            biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=True, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=False,
                        case=args_biotypes[iter_bio][-1])
            
            ## Bio Mask (for Training/Labeling)
            biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = True, BIOTOP_DESCRIPTION=False,
                        case=args_biotypes[iter_bio][-1])
            
            if args_text: create_info_textfile(temp_biotop)
        else:
            print("Already Existing Biotop: ", i+old_i, "/", len_totoal, " ", bio_i)
        
    old_i = old_i + i
print("Done.")
driver.quit()

Load: bmaporthofoto30cm
Start: webdriver  chrome
Already Existing Biotop:  0 / 646   564050102
Already Existing Biotop:  1 / 646   564050115
Already Existing Biotop:  2 / 646   565380128
Already Existing Biotop:  3 / 646   565380129
Already Existing Biotop:  4 / 646   564040266
Already Existing Biotop:  5 / 646   564040314
Already Existing Biotop:  6 / 646   564040321
Already Existing Biotop:  7 / 646   564040315
Already Existing Biotop:  8 / 646   564040317
Already Existing Biotop:  9 / 646   564040319
Already Existing Biotop:  10 / 646   564040293
Already Existing Biotop:  11 / 646   564040290
Already Existing Biotop:  12 / 646   564040289
Already Existing Biotop:  13 / 646   564040318
Already Existing Biotop:  14 / 646   564040281
Already Existing Biotop:  15 / 646   564040299
Already Existing Biotop:  16 / 646   564040301
Already Existing Biotop:  17 / 646   564040265
Already Existing Biotop:  18 / 646   564040163
Already Existing Biotop:  19 / 646   564040201
Already Existing Biot

Already Existing Biotop:  197 / 646   565492955
Already Existing Biotop:  198 / 646   565242946
Already Existing Biotop:  199 / 646   565242945
Already Existing Biotop:  200 / 646   565492948
Already Existing Biotop:  201 / 646   565492949
Already Existing Biotop:  202 / 646   565015987
Already Existing Biotop:  203 / 646   565015975
Already Existing Biotop:  204 / 646   565016006
Already Existing Biotop:  205 / 646   565503071
Already Existing Biotop:  206 / 646   565505077
Already Existing Biotop:  207 / 646   565282571
Already Existing Biotop:  208 / 646   565492951
Already Existing Biotop:  209 / 646   565282997
Already Existing Biotop:  210 / 646   565283010
Already Existing Biotop:  211 / 646   565283019
Already Existing Biotop:  212 / 646   565283006
Already Existing Biotop:  213 / 646   565040008
Already Existing Biotop:  214 / 646   565040196
Already Existing Biotop:  215 / 646   565160129
Already Existing Biotop:  216 / 646   565300188
Already Existing Biotop:  217 / 646   56

Already Existing Biotop:  391 / 646   565180217
Already Existing Biotop:  392 / 646   565180220
Already Existing Biotop:  393 / 646   565180214
Already Existing Biotop:  394 / 646   565180202
Already Existing Biotop:  395 / 646   565180053
Already Existing Biotop:  396 / 646   565180234
Already Existing Biotop:  397 / 646   565180157
Already Existing Biotop:  398 / 646   565070184
Already Existing Biotop:  399 / 646   565070167
Already Existing Biotop:  400 / 646   565070056
Already Existing Biotop:  401 / 646   565070170
Already Existing Biotop:  402 / 646   565070169
Already Existing Biotop:  403 / 646   565070035
Already Existing Biotop:  404 / 646   565430178
Already Existing Biotop:  405 / 646   565030154
Already Existing Biotop:  406 / 646   565430189
Already Existing Biotop:  407 / 646   565030173
Already Existing Biotop:  408 / 646   565030184
Already Existing Biotop:  409 / 646   565030147
Already Existing Biotop:  410 / 646   565030186
Already Existing Biotop:  411 / 646   56

Already Existing Biotop:  598 / 646   565011712
Already Existing Biotop:  599 / 646   565492954
Already Existing Biotop:  600 / 646   565271238
Already Existing Biotop:  601 / 646   565040010
Already Existing Biotop:  602 / 646   565040039
Already Existing Biotop:  603 / 646   565040029
Already Existing Biotop:  604 / 646   565040199
Already Existing Biotop:  605 / 646   565040205
Already Existing Biotop:  606 / 646   565040089
Already Existing Biotop:  607 / 646   565040085
Already Existing Biotop:  608 / 646   565040035
Already Existing Biotop:  609 / 646   565160122
Already Existing Biotop:  610 / 646   565160058
Already Existing Biotop:  611 / 646   565160021
Already Existing Biotop:  612 / 646   565160142
Already Existing Biotop:  613 / 646   565160145
Already Existing Biotop:  614 / 646   565300210
Already Existing Biotop:  615 / 646   565410052
Already Existing Biotop:  616 / 646   565410048
Already Existing Biotop:  617 / 646   565410060
Already Existing Biotop:  618 / 646   56

## 3.2) Extract single biotope

In [13]:
#script_dir = os.path.dirname(__file__)
#script_dir = '/Users/flo/Proj/sat_hedge_detection/src/bio_generator'

args_shapefile = '../data/Biotopkartierung/Biotopkartierung.shp'
args_biotypes = ['8.1.1.1', '8.1.1.2']
args_wmts = 'http://maps.wien.gv.at/basemap/bmaporthofoto30cm/normal/google3857/{z}/{y}/{x}.jpeg'
args_text = True
args_output_dir = '../data/output_biotop_dir/'

##
target_biotope = '565030179'
##



print('Load: bmaporthofoto30cm')
wmts = args_wmts
#driver = webdriver.Firefox(executable_path='/usr/local/bin/geckodriver')

if args_driver == 'chrome':
    print('Start: webdriver ', args_driver)
    #driver = webdriver.Chrome(executable_path=args_driver_path)
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(args_driver_path, options=options)
else:
    print('Different webdriver... break')

transformer = Transformer.from_crs(31258, 4326, always_xy=True)  # 1frpm 2to

str_nummer = "\'" + target_biotope + "\'"
query_str = "Nummer==" + str(str_nummer)
temp_biotop = nReserve.query(query_str)
bio_i = target_biotope
temp_location = biotop_center(temp_biotop)
#print(temp_biotop)

temp_typ = temp_biotop["biotoptyp"]
temp_key = (temp_typ.keys()[0])

PATH_NEW = create_bio_path(target_biotope)
if PATH_NEW:
    print("Process Biotop: ", target_biotope, " type: ", temp_typ[temp_key])
    #m_temp = biotop_current_map(temp_location,text=getText(temp_biotop),BIOTOP_BORDER=False, BIOTOP_DESCRIPTION=False, case=1)

    ## Raw Bio Image
    biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=True,
                case=temp_typ[temp_key][-1])

    ## Red Border File
    biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=True, BIOTOP_MASK = False, BIOTOP_DESCRIPTION=False,
                case=temp_typ[temp_key][-1])

    ## Bio Mask (for Training/Labeling)
    biotop_current_map(temp_location, text=getText(temp_biotop), BIOTOP_BORDER=False, BIOTOP_MASK = True, BIOTOP_DESCRIPTION=False,
                case=temp_typ[temp_key][-1])

    if args_text: create_info_textfile(temp_biotop)
else:
    print("Already Existing Biotop: ", target_biotope)

print("Done.")
driver.quit()

Load: bmaporthofoto30cm
Start: webdriver  chrome
Process Biotop:  565030179  type:  8.1.1.2
Done.


In [ ]:
biotop_dict[0]